# Autoscaling Ray on Databricks and Apache Spark

With the release of **Ray 2.8.0**, we have enabled Ray auto-scaling with Ray on Databricks and Apache Spark. Below, we showcase the functionality by going through an example of hyper-parameter tuning for a deep learning model on the CIFAR dataset.

Ray Auto-scaling works with **DBR runtime 14+**, and the code has been tested with the following cluster configurations:

**Azure**: Driver NC6s_v3 and autoscaling with 4 worker nodes NC6s_v3.

## Install the Ray library and any other python Dependencies
Once specified you do not need to respecify the libraries during Ray initialization

In [0]:
%pip install ray['default,tune'] >=2.8.0

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-feature-store 0.14.1 requires pyspark<4,>=3.1.2, which is not installed.
tensorflow 2.11.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.0 which is incompatible.
tensorboard 2.11.0 requires protobuf<4,>=3.9.2, but you have protobuf 4.25.0 which is incompatible.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

## Start the ray cluster 
Use the Ray on spark API's to start the cluster refer to the [here](https://docs.ray.io/en/latest/cluster/vms/user-guides/community/spark.html?highlight=ray.util.spark#ray-on-spark-apis)  for more details on the parameters

In [0]:
from ray.util.spark import setup_ray_cluster, shutdown_ray_cluster


num_cpu_cores_per_worker = 4 # total cpu's present in each node
num_cpus_head_node = 4
num_gpu_per_worker = 1
num_gpus_head_node = 1

ray_conf = setup_ray_cluster(
  num_worker_nodes= 4,#this should be set max number of nodes the cluster is allowed to auto-scale
  num_cpus_head_node= num_cpus_head_node, #this should be set cores used in the driver node used for jobs
  num_gpus_head_node= num_gpus_head_node, #this only should be set for GPU enabled cluster 
  num_cpus_per_node=num_cpu_cores_per_worker, #this should be set cores added from each worker node 
  num_gpus_per_node=num_gpu_per_worker,#this should be set gpus added from each worker node 
  autoscale = True)


2023-11-06 21:18:42,105	INFO cluster_init.py:528 -- Ray head hostname 10.139.64.118, port 9124


2023-11-06 21:18:44,307	INFO usage_lib.py:416 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats` before starting the cluster. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.
2023-11-06 21:18:44,307	INFO scripts.py:744 -- Local node IP: 10.139.64.118
2023-11-06 21:18:46,380	SUCC scripts.py:781 -- --------------------
2023-11-06 21:18:46,380	SUCC scripts.py:782 -- Ray runtime started.
2023-11-06 21:18:46,380	SUCC scripts.py:783 -- --------------------
2023-11-06 21:18:46,380	INFO scripts.py:785 -- Next steps
2023-11-06 21:18:46,380	INFO scripts.py:788 -- To add another node to this Ray cluster, run
2023-11-06 21:18:46,380	INFO scripts.py:791 --   ray start --address='10.139.64.118:9124'
2023-11-06 21:18:46,380	INFO scripts.py:800 -- To connec

2023-11-06 21:19:02,138	INFO cluster_init.py:640 -- Ray head node started.
2023-11-06 21:19:02,143	INFO worker.py:1489 -- Connecting to existing Ray cluster at address: 10.139.64.118:9124...
2023-11-06 21:19:02,153	INFO worker.py:1664 -- Connected to Ray cluster. View the dashboard at 10.139.64.118:9137 


To monitor and debug Ray from Databricks, view the dashboard at 
 https://dbc-dp-984752964297111.cloud.databricks.com/driver-proxy/o/984752964297111/1023-112611-gamx0lyy/9137/


Open Ray Cluster Dashboard in a new tab

In [0]:
#Incase you want to restart the cluster use `shutdown_ray_cluster` this will not restart the interpretor or REPL
# shutdown_ray_cluster()

## Import all the libraries

In [0]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from filelock import FileLock
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
import ray
from ray import train, tune
from ray.train import Checkpoint
from ray.tune.schedulers import ASHAScheduler
import time

In [0]:
def load_data(data_dir="./data"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # We add FileLock here because multiple workers will want to
    # download data, and this may cause overwrites since
    # DataLoader is not threadsafe.
    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform)

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [0]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

##The Train function
Now it gets interesting, because we introduce some changes to the example from the [PyTorch documentation](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

The full code example looks like this:

In [0]:
def train_cifar(config,loc):
    
    print("num_cpus:",int(train.get_context().get_trial_resources().head_cpus))
    torch.set_num_threads(int(train.get_context().get_trial_resources().head_cpus))
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    # To restore a checkpoint, use `train.get_checkpoint()`.
    loaded_checkpoint = train.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
           model_state, optimizer_state = torch.load(os.path.join(loaded_checkpoint_dir, "checkpoint.pt"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(config['max_epoch']):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        # Here we save a checkpoint. It is automatically registered with
        # Ray Tune and can be accessed through `train.get_checkpoint()`
        # API in future iterations.
        os.makedirs(f"{loc}/mymodel", exist_ok=True)
        torch.save(
            (net.state_dict(), optimizer.state_dict()), f"{loc}/mymodel/checkpoint.pt")
        checkpoint = Checkpoint.from_directory(f"{loc}/mymodel/")
        train.report({"loss": (val_loss / val_steps),"try_gpu" : False, "accuracy": correct / total}, checkpoint=checkpoint)
    print("Finished Training")

In [0]:
def main(num_samples=10, max_num_epochs=10,
         grace_period=5,cpus_per_trial=1, 
         gpus_per_trial=0 , loc = '/dbfs/pj/ray/'):
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16]),
        "max_epoch":20
    }
    scheduler = ASHAScheduler(
        max_t=config['max_epoch'],
        grace_period=5,
        reduction_factor=2)
    
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar,loc = loc),
            resources={"cpu": cpus_per_trial, "gpu":gpus_per_trial }
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        run_config=train.RunConfig(
        storage_path=os.path.expanduser(loc),
        name="tune_checkpointing_location",
    ),
        param_space=config,
    )
    results = tuner.fit()
    
    best_result = results.get_best_result("loss", "min")

    print("Best trial config: {}".format(best_result.config))
    print("Best trial final validation loss: {}".format(
        best_result.metrics["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_result.metrics["accuracy"]))

    test_best_model(best_result)


In [0]:
def test_best_model(best_result):
    best_trained_model = Net(best_result.config["l1"], best_result.config["l2"])
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    print("Best trial test set accuracy: {}".format(correct / total))

In [0]:
# Run a CPU only Trial
main(num_samples=8, max_num_epochs=10,grace_period=5,cpus_per_trial=3, gpus_per_trial=0 , loc = '/dbfs/pj/ray/')

In [0]:
# Run a GPU only Trial
main(num_samples=8, max_num_epochs=10,grace_period=5,cpus_per_trial=1, gpus_per_trial=0.5 , loc = '/dbfs/pj/ray/')

2023-11-06 21:19:06,551	INFO worker.py:1354 -- Using address 10.139.64.118:9124 set in the environment variable RAY_ADDRESS
2023-11-06 21:19:06,551	INFO worker.py:1489 -- Connecting to existing Ray cluster at address: 10.139.64.118:9124...
2023-11-06 21:19:06,558	INFO worker.py:1664 -- Connected to Ray cluster. View the dashboard at 10.139.64.118:9137 
2023-11-06 21:19:06,816	INFO tune.py:220 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-11-06 21:19:06,819	INFO tune.py:595 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+----------------------------------------------------------------+
| Configuration for experiment     tune_checkpointing_location   |
+----------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator         |
| Scheduler                        AsyncHyperBandScheduler       |
| Number of trials                 8                             |
+----------------------------------------------------------------+

View detailed results here: /dbfs/pj/ray/tune_checkpointing_location
To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/tune_checkpointing_location`

Trial status: 8 PENDING
Current time: 2023-11-06 21:19:07. Total running time: 0s
Logical resource usage: 0/4 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:V100)
+----------------------------------------------------------------+
| Trial name                status             lr     batch_size |
+--------------------------------------------------

100%|██████████| 170498071/170498071 [00:01<00:00, 95982674.65it/s] 


(train_cifar pid=3953) Extracting /root/ray_results/tune_checkpointing_location/train_cifar_1e5c3_00001_1_batch_size=16,lr=0.0017_2023-11-06_21-19-07/data/cifar-10-python.tar.gz to /root/ray_results/tune_checkpointing_location/train_cifar_1e5c3_00001_1_batch_size=16,lr=0.0017_2023-11-06_21-19-07/data
(autoscaler +39s) Resized to 8 CPUs, 2 GPUs.
(train_cifar pid=3953) Files already downloaded and verified
(train_cifar pid=3953) num_cpus: 1
(autoscaler +41s) Adding 1 node(s) of type ray.worker.


(train_cifar pid=3953) /databricks/python/lib/python3.10/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
(train_cifar pid=3953)   warnings.warn(_create_warning_msg(


...

(train_cifar pid=2850, ip=10.139.64.113) [20,  2000] loss: 1.194
Trial status: 7 TERMINATED | 1 RUNNING
Current time: 2023-11-06 21:38:39. Total running time: 19min 32s
Logical resource usage: 1.0/8 CPUs, 0.5/2 GPUs (0.0/1.0 NODE_ID_AS_RESOURCE, 0.0/2.0 accelerator_type:V100)
Current best trial: 1e5c3_00001 with loss=1.2401041355133056 and params={'l1': 4, 'l2': 8, 'lr': 0.0017008417716143818, 'batch_size': 16, 'max_epoch': 20}
+---------------------------------------------------------------------------------------------------------------------+
| Trial name                status               lr     batch_size     iter     total time (s)      loss     accuracy |
+---------------------------------------------------------------------------------------------------------------------+
| train_cifar_1e5c3_00005   RUNNING      0.00117903              4       19           864.621    1.38908       0.5269 |
| train_cifar_1e5c3_00000   TERMINATED   0.00119966              2       10          100

(train_cifar pid=2850, ip=10.139.64.113) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/dbfs/pj/ray/tune_checkpointing_location/train_cifar_1e5c3_00005_5_batch_size=4,lr=0.0012_2023-11-06_21-19-07/checkpoint_000019)



Trial status: 8 TERMINATED
Current time: 2023-11-06 21:39:16. Total running time: 20min 8s
Logical resource usage: 1.0/8 CPUs, 0.5/2 GPUs (0.0/1.0 NODE_ID_AS_RESOURCE, 0.0/2.0 accelerator_type:V100)
Current best trial: 1e5c3_00001 with loss=1.2401041355133056 and params={'l1': 4, 'l2': 8, 'lr': 0.0017008417716143818, 'batch_size': 16, 'max_epoch': 20}
+---------------------------------------------------------------------------------------------------------------------+
| Trial name                status               lr     batch_size     iter     total time (s)      loss     accuracy |
+---------------------------------------------------------------------------------------------------------------------+
| train_cifar_1e5c3_00000   TERMINATED   0.00119966              2       10          1001.14     1.45024       0.4933 |
| train_cifar_1e5c3_00001   TERMINATED   0.00170084             16       20           321.879    1.2401        0.5481 |
| train_cifar_1e5c3_00002   TERMINATED   0.01

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Best trial test set accuracy: 0.5479
